In [2]:
from janome.tokenizer import Tokenizer
from collections import defaultdict
import pandas as pd
import itertools
import math

In [3]:
"""
形態素解析と単語数を数える
"""
def text2data(text_list):
    t = Tokenizer()
    vocab={}
    text_sum = 0
    for text in text_list:
        for token in t.tokenize(text):
            word = ""
            partOfSpeech = token.part_of_speech.split(',')[0]
            if partOfSpeech == u'名詞':
                word = token.surface
                text_sum += 1
                if word not in vocab:
                    vocab[word] = 1                
                else:
                    vocab[word] += 1                    
    return vocab,text_sum

In [4]:
# 教師データ (カテゴリ毎にファイルがある。)
train_file = ['./data/data_type01_main.csv','./data/data_type02_main.csv','./data/data_type03_main.csv']
category_num = len(train_file)

# テストデータ
#test_file = '.data/data_test.csv'


In [5]:
"""
1.カテゴリの出現確率 : p_c
学習データの各カテゴリーのrecipe名を総recipe数で割った値
"""
p_c = [1/3 for i in range(category_num)]
print(p_c)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


In [11]:
"""
2. カテゴリ内の単語出現率の計算
- 各カテゴリーの単語数 : text_sum
- カテゴリ内の単語出現率 : wordprob
- カテゴリごとの単語出現率リスト : category_wordprob


MEMO:
    ラプラススムージングスムージング追加 分子に1、分母に2
"""

category_wordprob = {}
category_textprob = {}
p_o = 0

for i,file_name in enumerate(train_file):

    # 教師データ用意
    df = pd.read_csv(file_name)
    text_list = df[:30].values.tolist()
    recipe_data = itertools.chain(*text_list)
    recipe_data = list(recipe_data)
    print(len(recipe_data))

    # 教師データを形態素解析して
    wordcount,text_sum = text2data(recipe_data)
    
    # カテゴリー内の単語出現率をもとめる
    wordprob = {}
    for k, v in wordcount.items():
        wordprob[k] = v/(text_sum+2)
    category_wordprob[i] = wordprob


100
100
100


In [10]:
"""
3. テストデータを分類する
カテゴリ内でのテストデータの出現率の確率を求め、テストデータのカテゴリー出現率の計算する
- 
- カテゴリ内の文書出現率 : category_textprob

"""

# テストデータ
test_file = './data/data_test.csv'
# テストデータ用意
df = pd.read_csv(test_file)
test_data = df.values.tolist()

tets_num = len(test_data)
success_num = 0
exception = 0
a = [0 for i in range(category_num)] # カテゴリに属するデータをカテゴリと分類した数
b = [0 for i in range(category_num)] 
c = [0 for i in range(category_num)]

# テストデータ分の分類を行う
for i,data in enumerate(test_data):
    
    test_text = [] # テストデータのリスト (データOの集合)
    category_textprob = [1 for i in range(category_num)]
    category_id = data[1]
    p_o = 0
    
    # テストデータを形態素解析
    t = Tokenizer()
    for token in t.tokenize(data[0]):
        word = ""
        partOfSpeech = token.part_of_speech.split(',')[0]
        if partOfSpeech == u'名詞':   
            word = token.surface
            if word not in test_text:
                test_text.append(word)
    
    
    print(str(i)+"回目",test_text,"カテゴリ："+str(category_id))
    
    # カテゴリーごとに計算
    for j in range(category_num): 

        # カテゴリー内のレシピ名の出現率 : p(o|c) 
        for test_word in test_text:        
            if test_word in category_wordprob[j]:
                category_textprob[j] *=  category_wordprob[j][test_word]               
            else:
                category_textprob[j] *= 1/(text_sum+2)                                
        
        # レシピ名の得られる確率
        p_o += category_textprob[j] * p_c[j]        
    
    ans_list = []
    for j in range(category_num):               
        ans_list.append((category_textprob[j]*p_c[j])/p_o)        
        
    # 正解率    
    max_ans = ans_list.index(max(ans_list))
    if ans_list[max_ans] == 0.3333333333333333:
        exception += 1
    elif max_ans == data[1]: # カテゴリに属するデータをカテゴリと分類した数
        success_num += 1
        a[category_id] += 1
    else: # カテゴリに属さないデータをカテゴリと分類した数
        b[max_ans] += 1

# 結果出力
print("正解率："+ str(success_num/tets_num))
print("該当なし："+ str(exception))
for i in range(category_num):
    print("----------------------------------")
    print("カテゴリid："+str(i))
    recall = a[i]/11
    print("再現率："+ str(recall))
    precision = a[i]/(a[i]+b[i])
    print("精度："+ str(precision))
    f = (2 * precision * recall)/(precision + recall)
    print("F値：" + str(f))


0回目 ['キノコ', 'ベーコン', '和風', 'パスタ'] カテゴリ：0
6.045825869938602e-10
0 0.9690437766952193
1 0.0046988809191968305
2 0.026257342385583946
1回目 ['ジャコ', 'ご飯'] カテゴリ：0
4.643742881671861e-05
0 0.9838214807507242
1 0.00202231490615947
2 0.01415620434311629
2回目 ['チーズフォカッチャ'] カテゴリ：0
0.0005307855626326963
0 0.33333333333333337
1 0.33333333333333337
2 0.33333333333333337
3回目 ['ひき肉', '丼'] カテゴリ：0
1.6548614461787205e-05
0 0.8444564362999553
1 0.14986870163128235
2 0.005674862068762414
4回目 ['春野', '菜', 'スープ', '丼'] カテゴリ：0
2.2874952937694766e-11
0 0.9850663283253366
1 0.011463777762684943
2 0.0034698939119783964
5回目 ['中華', '風', 'そぼろ', '丼'] カテゴリ：0
2.4789682070766013e-09
0 0.9847292376055348
1 0.015185378849587022
2 8.538354487829779e-05
6回目 ['ジンジャー', '焼きそば'] カテゴリ：0
8.986682507167168e-07
0 0.7974525999548345
1 0.0980470970413905
2 0.10450030300377501
7回目 ['キーマカレー'] カテゴリ：0
0.0006994592536092867
0 0.4940991345397325
1 0.25295043273013373
2 0.25295043273013373
8回目 ['ごはん'] カテゴリ：0
0.0017362658891717543
0 0.79619651347

In [ ]:
category_textprob = {}
p_o = 0

for 
# カテゴリー内の文書出現率 : p(o|c)    
    if "ヒジキ" in wordprob and "和風" in wordprob:
        textprob[i] = wordprob["ヒジキ"] * wordprob["和風"]        
        p_o += wordprob["ヒジキ"] * wordprob["和風"] * p_c[i]
    else:
        textprob[i] = 0    

for i in range(category_num):               
    print(i,(textprob[i]*p_c[i])/p_o)